# 🎨 NeMo Data Designer: Japanese Commonsense Reasoning Dataset Generation

#### 📚 Overview

This notebook generates synthetic datasets for the following tasks using NeMo Data Designer:
- **jcommonsenseqa**: Japanese commonsense question answering

**Seed Data**: Uses `nvidia/Nemotron-Personas-Japan` directly as the dataset

<br>

> 👋 **Important** – Environment Setup
>
> - Ensure that NeMo Data Designer installation and configuration are completed
> - Ensure that the local LLM server is running


### 📦 Import Required Modules

In [1]:
from nemo_microservices.data_designer.essentials import (
    CategorySamplerParams,
    DataDesignerConfigBuilder,
    ExpressionColumnConfig,
    InferenceParameters,
    LLMJudgeColumnConfig,
    LLMStructuredColumnConfig,
    LLMTextColumnConfig,
    ModelConfig,
    NeMoDataDesignerClient,
    SamplerColumnConfig,
    SamplerType,
    Score,
)

from pydantic import BaseModel, Field
from datasets import load_dataset
import pandas as pd
import random

### ⚙️ Initialize NeMo Data Designer Client

In [2]:
NEMO_MICROSERVICES_BASE_URL = "http://localhost:8080"

data_designer_client = NeMoDataDesignerClient(base_url=NEMO_MICROSERVICES_BASE_URL)

### 🎛️ Define Model Configuration

In [70]:
MODEL_PROVIDER = "nvidiabuild"
MODEL_ID = "openai/gpt-oss-120b"
MODEL_ALIAS = "gpt-oss-120b"
SYSTEM_PROMPT = ""  
JUDGE_MODEL_ALIAS = "quality-judge"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.9,
            top_p=0.95,
            max_tokens=2048,
            max_parallel_requests=8,
            timeout=1200
        ),
    ),
    ModelConfig(
        alias=JUDGE_MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.3,
            top_p=0.9,
            max_tokens=1024,
            max_parallel_requests=4,
            timeout=1500,
        ),
    ),
]

### 📊 Prepare Seed Data

Load persona data from `nvidia/Nemotron-Personas-Japan` and
pass it to Data Designer as a pandas DataFrame.

In [4]:
import json
import pandas as pd

personas_dataset = load_dataset("nvidia/Nemotron-Personas-Japan", split="train")

# DataFrameに変換
df = personas_dataset.to_pandas()

## Define Target Count and Category Breakdown

Define target of 2000 total seeds with category-specific breakdowns.

- **SEED_TARGET**: 2000 total seeds
- **WeakA**: geo(250), tools(100), public(200), other(150) = 400 total
- **WeakB** (weakness reinforcement): finance(400), safety(350), vocab(350) = 1100 total
- **Typical**: Remaining 500 seeds
- **Bias suppression**: Max 10 per occupation, max 12 per prefecture

In [5]:
import re
import numpy as np
import pandas as pd
from typing import List, Optional

np.random.seed(42)


SEED_TARGET = 2000

# weakB（弱点補強）: 合計1100
WEAKB_TARGETS = {"finance": 400, "safety": 350, "vocab": 350}
N_WEAK_A = 400        # weakA全体
N_TYPICAL = SEED_TARGET - sum(WEAKB_TARGETS.values()) - N_WEAK_A  # = 500

# weakA 内をサブクォータで固定（ここが最重要）
N_GEO = 250
N_TOOLS = 100
N_PUBLIC = 200
N_WEAK_A_OTHER = N_WEAK_A - N_GEO - N_TOOLS - N_PUBLIC  # 150

# 偏り抑制（cap）
CAP_PER_OCCUPATION = 10
CAP_PER_PREFECTURE = 12

# D(公共)を“宗教/儀礼”に吸わせないための優先度（public_bonus）
PUBLIC_BONUS_KW = ["図書館","病院","役所","窓口","待合室","列","並ぶ","順番","受付","会計",
                   "駅","改札","切符","ゴミ","分別","ルール","禁止","優先席","エスカレーター","エレベーター"]
RELIGION_PENALTY_KW = ["寺","神社","教会","礼拝堂","ミサ","修道院","仏壇","お経","線香","数珠","墓","お守り"]

# 再現性: hash(cat)禁止 → 固定seed
CAT_RS = {"finance": 201, "safety": 202, "vocab": 203}
WA_RS  = {"geo": 111, "tools": 112, "other": 113}
TY_RS  = 301

# neutral を増やしすぎない（上限）
NEUTRAL_CAP = 50

## Handle Missing Values

Fill missing values in required columns with empty strings.
Create columns with empty strings if they don't exist.

## Text Construction

Combine multiple columns to construct text for classification.

- `_all_text`: Combine all columns
- `_core_text`: Combine core columns only (primary target for keyword matching)
- `_core_len`: Character count of core text

In [6]:

TEXT_COLS_ALL = [
    "occupation","hobbies_and_interests","marital_status","education_level","prefecture","region","area",
    "professional_persona","cultural_background","travel_persona","culinary_persona","persona",
]
CORE_COLS = ["occupation","hobbies_and_interests","marital_status","education_level","prefecture","region","area"]
EXTRA_COLS = ["uuid","age","age_band","skills_and_expertise_list",
              "travel_persona","hobbies_and_interests","area"]

for col in sorted(set(TEXT_COLS_ALL + CORE_COLS + EXTRA_COLS)):
    if col in df.columns:
        df[col] = df[col].fillna("").astype(str)
    else:
        df[col] = ""

def build_text(row: pd.Series, cols: List[str]) -> str:
    parts = [row.get(c, "") for c in cols]
    parts = [p for p in parts if str(p).strip()]
    return " / ".join(parts)

df["_all_text"]  = df.apply(lambda r: build_text(r, TEXT_COLS_ALL), axis=1)
df["_core_text"] = df.apply(lambda r: build_text(r, CORE_COLS), axis=1)
df["_core_len"]  = df["_core_text"].str.len()

## Create Duplicate Suppression Key

Create a key (`_attr_key`) for duplicate detection based on attribute combinations.
This prevents selecting multiple similar personas.

Exclude completely empty keys (all fields empty).

In [7]:

KEY_COLS = ["prefecture","region","area","occupation","education_level","marital_status"]

def _norm(s: str) -> str:
    s = "" if s is None else str(s)
    return re.sub(r"\s+", " ", s.strip())

df["_attr_key"] = df.apply(lambda r: "|".join([_norm(r.get(c, "")) for c in KEY_COLS]), axis=1)
df = df[df["_attr_key"] != "|||||"].copy()

## Exclude by Negative Keywords

Exclude personas containing inappropriate keywords (extreme expressions, crime-related, etc.) unsuitable for JCommonsenseQA.
Evaluate `_core_text` and remove matching entries.

In [8]:
NEG_KW_JC = [
    "陰謀","反ワク","極右","極左","ヘイト","差別","排外","テロ","過激",
    "万引き","窃盗","詐欺","横領","覚醒剤","麻薬",
    "絶対","許せない","嫌悪","憎い",
]
df["has_neg_jc"] = df["_core_text"].str.contains("|".join(map(re.escape, NEG_KW_JC)), regex=True, na=False)
df_jc = df[~df["has_neg_jc"]].copy()

## Define Category Keyword Dictionaries

Define keywords for transportation/movement, daily life/housework, and tools.

- **geo_kw**: A_Transportation/Movement (trains, stations, buses, walking, etc.)
- **life_kw**: F_Daily Life/Housework (cooking, cleaning, shopping, etc.)
- **tools_kw**: B_Tools/Usage (knives, vacuum cleaners, stationery, etc.)

Define keywords for public facilities/manners, culture/etiquette, and finance.

- **public_kw**: D_Public Facilities/Manners (lines, order, priority seats, etc.)
- **culture_kw**: D_Public Facilities/Manners (etiquette, ceremonies, etc.)
- **finance_kw**: C_Payment/Money (accounting, banking, card payments, etc.)

In [9]:
geo_kw = ["電車","地下鉄","新幹線","駅","改札","切符","定期券","ICカード","Suica","PASMO","バス","バス停","タクシー",
          "高速道路","駐車場","乗り換え","時刻表","終電","始発","渋滞","踏切","信号","ホーム","路線","乗車券",
          "徒歩","歩く","歩道","横断歩道","交差点","信号待ち","右折","左折","移動","目的地","経路","ルート",
          "地図","ナビ","最寄り","乗る","降りる","下車","乗車","入口","出口","改札口","ホームドア","階段","乗り場"]

life_kw = ["料理","掃除","片付け","DIY","買い物","健康","睡眠","育児","子育て","弁当","洗濯","家事",
           "ゴミ出し","分別","資源ごみ","可燃","不燃","粗大ごみ","節約","整理整頓","収納",
           "病院","薬局","会計","レジ",
           "戸棚","食器棚","本棚","引き出し","クローゼット","タンス","棚","机","椅子",
           "掃除機","洗濯機","電子レンジ","炊飯器"]

tools_kw = ["包丁","まな板","鍋","フライパン","菜箸","おたま","ほうき","ちりとり","雑巾","スポンジ","洗剤","漂白剤",
            "掃除機","ドライバー","ペンチ","金づち","かなづち","ノコギリ","シャベル","釘","ハサミ",
            "ペン","鉛筆","シャーペン","消しゴム","ノート","付箋","ホッチキス","クリップ","定規","マーカー","カッター",
            "ガムテープ","セロテープ","ラップ","アルミホイル","トング","計量スプーン","計量カップ",
            "延長コード","充電器","電池","ドライヤー","アイロン"]

public_kw = ["公共","ルール","順番","列","並ぶ","割り込み","優先席","禁煙","喫煙","騒音","静かに","ゴミ","ポイ捨て","迷惑",
             "図書館","映画館","病院","待合室","エスカレーター","エレベーター","コンビニ","スーパー","店内"]

culture_kw = ["礼儀","作法","冠婚葬祭","お辞儀","正月","お盆","祭り","年中行事","着物","茶道",
              "挨拶","敬語","名刺","手土産","お礼","お詫び","謝罪","断る","遠慮","失礼","配慮","気遣い",
              "神社","寺","仏教","神道"]

finance_kw = ["支払い","会計","料金","値段","割引","クーポン","精算","返金","領収書","レシート","釣銭","おつり",
              "銀行","ATM","口座","振込","送金","引き落とし","請求","請求書","利用明細","明細","手数料","ポイント",
              "クレジットカード","デビット","電子マネー","キャッシュレス","税","納付","控除","保険","ローン","分割払い","家計","予算"]

safety_kw = ["危険","危ない","事故","転倒","火","火事","火傷","やけど","ガス","ガス漏れ","一酸化炭素","換気",
             "感電","漏電","ガソリン","発火","刃物","ナイフ","包丁",
             "飲酒運転","信号無視","スピード","雪道","凍結","ノーマルタイヤ","ヘルメット","シートベルト",
             "漂白剤","殺虫剤","洗剤","薬","誤飲","食中毒","熱中症","凍傷",
             "避難","非常口","消火器","火災報知器","警報","防災"]

vocab_kw = ["用途","目的","分類","種類","意味","定義","言い換え","言い回し","表現",
            "類義語","対義語","反対語","ことわざ","慣用句","比喩","例え",
            "適切","不適切","一般的","代表的","日常的"]

JC_CAT_KWS = {
    "finance": finance_kw,
    "safety":  safety_kw,
    "vocab":   vocab_kw,
    "public":  public_kw,
    "tools":   tools_kw,
    "life":    life_kw,
    "geo":     geo_kw,
    "culture": culture_kw,
}

def count_hits(text: str, kws: List[str]) -> int:
    if not text:
        return 0
    return sum(1 for k in kws if k in text)

ALL_JC_KW = [kw for kws in JC_CAT_KWS.values() for kw in kws]

## Calculate Keyword Scores

Score how many keywords from each category are contained in the persona text.

- Primarily calculate scores using `_core_text`
- **geo/tools** only: recalculate by adding supplementary text (travel_persona, hobbies, etc.)
  - This prevents depletion of geo and tools categories and suppresses misclassification to public
 
## Exclude Abnormal Scores and Estimate Categories

Exclude data with abnormally high scores (containing unnaturally many keywords) and estimate the most suitable category for each persona.

- Select the category with the highest score
- In case of ties, decide by priority (finance > safety > vocab > ...)

In [10]:
# 基本は core_text
for cat, kws in JC_CAT_KWS.items():
    df_jc[f"score_{cat}"] = df_jc["_core_text"].map(lambda t, _kws=kws: count_hits(t, _kws))

# geo/tools のみ補助テキストで再計算（限定列のみ足す）
df_jc["_geo_text"] = df_jc["_core_text"] + " / " + df_jc["travel_persona"] + " / " + df_jc["area"]
df_jc["_tools_text"] = df_jc["_core_text"] + " / " + df_jc["hobbies_and_interests"] + " / " + df_jc["skills_and_expertise_list"]

df_jc["score_geo"] = df_jc["_geo_text"].map(lambda t: count_hits(t, geo_kw))
df_jc["score_tools"] = df_jc["_tools_text"].map(lambda t: count_hits(t, tools_kw))

df_jc["_kw_hits"] = df_jc["_core_text"].map(lambda t: count_hits(t, ALL_JC_KW))

MAX_SCORE = {"finance":14,"safety":16,"vocab":12,"public":10,"tools":10,"life":10,"geo":10,"culture":10}
mask_ok = pd.Series(True, index=df_jc.index)
for c, mx in MAX_SCORE.items():
    mask_ok &= (df_jc[f"score_{c}"] <= mx)
df_jc = df_jc[mask_ok].copy()

def assign_jc_category(row: pd.Series) -> Optional[str]:
    scores = {c: row[f"score_{c}"] for c in JC_CAT_KWS}
    maxv = max(scores.values()) if scores else 0
    if maxv <= 0:
        return None
    tied = [c for c, v in scores.items() if v == maxv]
    priority = ["finance","safety","vocab","public","tools","life","geo","culture"]
    for p in priority:
        if p in tied:
            return p
    return tied[0]

df_jc["jc_category"] = df_jc.apply(assign_jc_category, axis=1)

## Determine Neutral Data

Classify personas with few keyword hits and shorter length as Neutral.

**Conditions:**
- Core text length is 260 characters or less
- Keyword hit count is 0
- Does not contain definition keywords (such as '~とは')

Limit Neutral to 50 entries to prevent too many thin seeds.

## Create Sampling Pools

Create sampling pools for each category.

- **typical_pool**: Neutral and thin data (max_score ≤ 2)
- **weakB_pool**: Reinforcement targets (finance, safety, vocab)
- **geo_pool, tools_pool, public_pool, other_pool**: Each sub-category of WeakA

In [11]:
DEFINITION_KW = ["とは","という","意味","定義","何という","何と言う","どれを指す"]
df_jc["_has_definition"] = df_jc["_core_text"].str.contains("|".join(map(re.escape, DEFINITION_KW)), regex=True, na=False)

NEUTRAL_MAX_LEN_CORE = 260
NEUTRAL_MAX_HITS = 0
df_jc["is_neutral"] = (
    (df_jc["_core_len"] > 0)
    & (df_jc["_core_len"] <= NEUTRAL_MAX_LEN_CORE)
    & (df_jc["_kw_hits"] <= NEUTRAL_MAX_HITS)
    & (~df_jc["_has_definition"])
)

neutral_pool = df_jc[df_jc["is_neutral"]].drop_duplicates(subset=["_attr_key"]).copy()
neutral_pool["jc_category"] = "neutral"
neutral_pool = neutral_pool.sort_values("_core_len").head(NEUTRAL_CAP)  # ★上限

jc_pool = df_jc[df_jc["jc_category"].notna()].drop_duplicates(subset=["_attr_key"]).copy()

score_cols = [f"score_{c}" for c in JC_CAT_KWS]
jc_pool["max_score_any"] = jc_pool[score_cols].max(axis=1)

typical_pool = (
    pd.concat([neutral_pool, jc_pool[jc_pool["max_score_any"] <= 2]], axis=0)
    .drop_duplicates(subset=["_attr_key"])
    .copy()
)

weakB_pool = jc_pool[jc_pool["jc_category"].isin(["finance","safety","vocab"])].copy()
# weakAはここで分割して使う（geo/tools/other）
geo_pool   = jc_pool[jc_pool["jc_category"]=="geo"].copy()
tools_pool = jc_pool[jc_pool["jc_category"]=="tools"].copy()
public_pool = jc_pool[jc_pool["jc_category"]=="public"].copy()  # ★追加
other_pool  = jc_pool[jc_pool["jc_category"].isin(["culture","life"])].copy()  # ★publicを除外

## Define Sampling Function with Caps

Function that samples while suppressing bias by occupation and prefecture.

**Operation:**
1. First sample while respecting caps
2. If insufficient, relax caps to fill the remainder
3. Always ensure the specified count is met

In [12]:
def sample_with_caps(pool: pd.DataFrame, n: int, random_state: int = 0) -> pd.DataFrame:
    if n <= 0 or len(pool) == 0:
        return pool.iloc[0:0].copy()

    pool2 = pool.sample(frac=1.0, random_state=random_state).copy()

    out = []
    occ_cnt, pref_cnt = {}, {}

    # 1st pass: capあり
    for _, r in pool2.iterrows():
        occ = str(r.get("occupation",""))
        pref = str(r.get("prefecture",""))
        if occ and occ_cnt.get(occ, 0) >= CAP_PER_OCCUPATION:
            continue
        if pref and pref_cnt.get(pref, 0) >= CAP_PER_PREFECTURE:
            continue
        out.append(r)
        occ_cnt[occ] = occ_cnt.get(occ, 0) + 1
        pref_cnt[pref] = pref_cnt.get(pref, 0) + 1
        if len(out) >= n:
            return pd.DataFrame(out)

    # 2nd pass: cap緩和（残りを素直に埋める）
    used_keys = set([r["_attr_key"] for r in out if "_attr_key" in r])
    for _, r in pool2.iterrows():
        k = r.get("_attr_key", None)
        if k is not None and k in used_keys:
            continue
        out.append(r)
        if len(out) >= n:
            break

    return pd.DataFrame(out)

## Sample WeakB Categories

Sample weakness reinforcement targets (WeakB).

- **finance**: 400 entries
- **safety**: 350 entries
- **vocab**: 350 entries

Sample a total of 1100 entries, excluding already selected data from subsequent sampling.

## WeakA - Sample Geo/Tools

Sample transportation/movement and tools categories from WeakA.

- **geo** (Transportation/Movement): 250 entries
- **tools** (Tools): 100 entries

## WeakA - Sample Public/Other

Sample the remainder of WeakA.

- **public** (Public Facilities/Manners): 200 entries
- **other** (culture/life): 150 entries
  - For Other, prioritize those with public facility-related keywords
  - Suppress those with religion-related keywords (penalty)
  - This prevents category D from being biased toward religion

## Sample Typical and Final Adjustments

Fill the remaining slots (approx. 500 entries) from the Typical category.

**Process:**
1. Sample remaining count from Typical pool
2. Combine all parts
3. If insufficient, add from unused data
4. If excess, adjust to 2000 entries
5. **Always ensure exactly 2000 entries**

In [13]:
seed_parts = []
used = set()

# ---- weakB（固定内訳）----
for cat, n in WEAKB_TARGETS.items():
    g = weakB_pool[(weakB_pool["jc_category"] == cat) & (~weakB_pool["_attr_key"].isin(used))].copy()
    s = sample_with_caps(g, min(n, len(g)), random_state=CAT_RS[cat])
    seed_parts.append(s)
    used |= set(s["_attr_key"].tolist())

# ---- weakA（サブクォータ）----
g = geo_pool[~geo_pool["_attr_key"].isin(used)].copy()
s = sample_with_caps(g, min(N_GEO, len(g)), random_state=WA_RS["geo"])
seed_parts.append(s)
used |= set(s["_attr_key"].tolist())

g = tools_pool[~tools_pool["_attr_key"].isin(used)].copy()
s = sample_with_caps(g, min(N_TOOLS, len(g)), random_state=WA_RS["tools"])
seed_parts.append(s)
used |= set(s["_attr_key"].tolist())

g = public_pool[~public_pool["_attr_key"].isin(used)].copy()
s = sample_with_caps(g, min(N_PUBLIC, len(g)), random_state=WA_RS.get("public", 114))
seed_parts.append(s)
used |= set(s["_attr_key"].tolist())

g = other_pool[~other_pool["_attr_key"].isin(used)].copy()
# public優先・宗教抑制を効かせる
def _count_kw(text: str, kws) -> int:
    if not text:
        return 0
    return sum(1 for k in kws if k in text)

g["_public_bonus"] = g["_core_text"].map(lambda t: _count_kw(t, PUBLIC_BONUS_KW))
g["_religion_pen"] = g["_core_text"].map(lambda t: _count_kw(t, RELIGION_PENALTY_KW))
g = g.sort_values(["_public_bonus","_religion_pen","_core_len"], ascending=[False, True, True]).copy()
s = sample_with_caps(g, min(N_WEAK_A_OTHER, len(g)), random_state=WA_RS["other"])
seed_parts.append(s)
used |= set(s["_attr_key"].tolist())

# ---- typical（残り）----
remain = SEED_TARGET - sum(len(x) for x in seed_parts)
g = typical_pool[~typical_pool["_attr_key"].isin(used)].sort_values("_core_len", ascending=True).copy()
s = sample_with_caps(g, min(remain, len(g)), random_state=TY_RS)
seed_parts.append(s)

seed_jc = pd.concat(seed_parts, axis=0).drop_duplicates(subset=["_attr_key"]).copy()

# サイズを必ず合わせる（最後に埋め戻し）
remain = SEED_TARGET - len(seed_jc)
if remain > 0:
    pool = df_jc[~df_jc["_attr_key"].isin(set(seed_jc["_attr_key"]))].drop_duplicates(subset=["_attr_key"]).copy()
    if len(pool) == 0:
        raise ValueError("fill pool empty; cannot reach SEED_TARGET")
    seed_jc = pd.concat([seed_jc, pool.head(remain)], axis=0).drop_duplicates(subset=["_attr_key"]).copy()

# 最終調整
if len(seed_jc) > SEED_TARGET:
    seed_jc = seed_jc.sample(n=SEED_TARGET, random_state=123)
elif len(seed_jc) < SEED_TARGET:
    raise ValueError(f"seed pool too small: seed_jc={len(seed_jc)} < {SEED_TARGET}")

## Assign Themes and Check Distribution

Map categories to JCommonsenseQA themes (A-F, N) and check the final distribution.

**Themes:**
- A: Transportation/Movement
- B: Tools/Usage
- C: Payment/Money
- D: Public Facilities/Manners
- E: Safety/Danger
- F: Daily Life/Housework
- N: Neutral

In [14]:
# ============================================================
# 9) jc_theme（categoryから決め打ち）
# ============================================================
CAT2THEME = {
    "finance": "C_支払い・お金",
    "safety":  "E_安全・危険",
    "vocab":   "B_道具・用途",
    "tools":   "B_道具・用途",
    "public":  "D_公共施設・マナー手順",
    "culture": "D_公共施設・マナー手順",
    "geo":     "A_交通・移動",
    "life":    "F_生活・家事",
    "neutral": "N_ニュートラル",
}
seed_jc["jc_theme"] = seed_jc["jc_category"].map(CAT2THEME).fillna("N_ニュートラル")

print("[seed_jc] size:", len(seed_jc))
print(seed_jc["jc_category"].value_counts(dropna=False))
print(seed_jc["jc_theme"].value_counts(dropna=False))


[seed_jc] size: 2000
jc_category
finance    407
vocab      364
safety     352
geo        320
public     208
culture    145
tools      102
life       102
Name: count, dtype: int64
jc_theme
B_道具・用途         466
C_支払い・お金        407
D_公共施設・マナー手順    353
E_安全・危険         352
A_交通・移動         320
F_生活・家事         102
Name: count, dtype: int64


## Create and Save Final Output Data

Select columns needed for prompt generation and create the final seed data.

**Output Columns:**
- uuid, occupation, prefecture, region, marital_status
- age_band, skills_and_expertise_list
- jc_theme, jc_category, _attr_key


In [15]:
# ============================================================
# 10) seed_out（prompt整合）
# ============================================================
required_cols = ["occupation","prefecture","region","marital_status","age_band","jc_theme","skills_and_expertise_list"]
for c in required_cols:
    if c not in seed_jc.columns:
        seed_jc[c] = ""

# age/skills を使わない方針なら、ここで空にする（推奨）
# seed_jc["age_band"] = ""
# seed_jc["skills_and_expertise_list"] = ""

seed_jc["skills_and_expertise_list"] = seed_jc["skills_and_expertise_list"].fillna("").astype(str)

seed_out_cols = [
    "uuid","occupation","prefecture","region","marital_status",
    "age_band","skills_and_expertise_list","jc_theme","jc_category","_attr_key"
]
seed_out_cols = [c for c in seed_out_cols if c in seed_jc.columns]
seed_out = seed_jc[seed_out_cols].copy()

seed_out.head(10)


,uuid,occupation,prefecture,region,marital_status,age_band,skills_and_expertise_list,jc_theme,jc_category,_attr_key
836023,749db6e7c2e245b2ae3b46aa12c4f1e0,保険業 中小,三重県,近畿地方,離別 (子供あり),,"['保険契約管理', 'リスク評価', '顧客対応', '法規制遵守', 'Excel業務']",C_支払い・お金,finance,三重県|近畿地方|西日本|保険業 中小|中学卒|離別 (子供あり)
333198,e8d5c500090946b586bd969f84f8ca78,保険業 中小,大阪府,近畿地方,既婚,,"['保険提案・契約手続き', 'リスク評価', '顧客信頼構築', '法令遵守チェック', ...",C_支払い・お金,finance,大阪府|近畿地方|西日本|保険業 中小|高校卒|既婚
633465,07f13df066824f33ba51dac21eafccf4,保険業 中堅,鳥取県,中国地方,未婚,,"[""リスク分析"", ""保険商品知識"", ""顧客対応"", ""書類管理"", ""法令遵守"", ""チ...",C_支払い・お金,finance,鳥取県|中国地方|西日本|保険業 中堅|大学卒 文系|未婚
505505,74f03ab6afbe4ba1b56f727c8aca5884,保険業 大手 (現在は引退),三重県,近畿地方,死別,,"[""保険リスク評価"",""顧客対応・カウンセリング"",""制度遵守・コンプライアンス"",""資産設...",C_支払い・お金,finance,三重県|近畿地方|西日本|保険業 大手 (現在は引退)|高専卒|死別
136573,cb1384fa55294c888630952ee92c488c,保険業 中堅 (現在は引退),愛知県,中部地方,死別 (子供あり),,"['リスク評価','顧客対応','契約管理','Excel','メール','保険オンラインポ...",C_支払い・お金,finance,愛知県|中部地方|東日本|保険業 中堅 (現在は引退)|大学卒 理系|死別 (子供あり)
12885,32ab728886554d89a04ee7ea7d42a3cd,保険業 大手,京都府,近畿地方,既婚,,"['保険商品知識', 'リスク分析', 'チームマネジメント', 'Excel', 'Pow...",C_支払い・お金,finance,京都府|近畿地方|西日本|保険業 大手|大学卒 文系|既婚
112485,c20c606f4b864e1db4fdd379299409e2,保険業 中堅 (現在は引退),東京都,関東地方,死別,,"['保険契約のリスク評価', '顧客対応と信頼関係構築', '保険商品に関する法規遵守', ...",C_支払い・お金,finance,東京都|関東地方|東日本|保険業 中堅 (現在は引退)|高校卒|死別
110168,395ad83f309343a7adef1ed7d3e1b3f0,保険業 中小,千葉県,関東地方,既婚,,"['保険商品知識','顧客対応','リスク分析','営業提案','書類作成','チームリーダ...",C_支払い・お金,finance,千葉県|関東地方|東日本|保険業 中小|高校卒|既婚
419644,8f2df7cef4bd42c884f965f310423221,保険業 中堅,香川県,四国地方,既婚 (子供あり),,"['保険商品知識','顧客対応','契約手続き管理','Microsoft Office活用...",C_支払い・お金,finance,香川県|四国地方|西日本|保険業 中堅|高校卒|既婚 (子供あり)
604760,94f5b775f7264e9a872baf7d2ec8e25e,保険業 中堅 (現在は引退),滋賀県,近畿地方,死別,,"[""リスク評価と契約審査"", ""シニア資産運用アドバイス"", ""金融法規遵守"", ""地域金融...",C_支払い・お金,finance,滋賀県|近畿地方|西日本|保険業 中堅 (現在は引退)|大学卒 理系|死別


## Theme-Based Topic Category Assignment

This code probabilistically assigns **topic_category** to each seed persona based on JCommonsenseQA themes (A-F, N).

### Main Features

1. **Define weights for each theme**
   - Define assignment probabilities for topic categories (transportation, public places, daily life, etc.) for each theme (transportation/movement, tools/usage, etc.)
   - Example: `A_Transportation/Movement` → Transportation 60%, Public places 25%, Daily life 15%

2. **Deterministic random number generation**
   - `stable_u01()`: Always generates the same 0-1 value from UUID/attribute key (using MD5 hash)
   - Guarantees complete reproducibility as it always returns the same result for the same input

3. **Weighted probability selection**
   - `pick_weighted()`: Selects categories based on weights using cumulative probability method
   - Determines appropriate topic category based on 0-1 random value

4. **Automatic key selection**
   - Uses `uuid` column if it exists, otherwise uses `_attr_key`
   - Identifier for assigning unique stable random numbers to each row

### Process Flow
```
Row data → Get jc_theme → Get weights for theme 
→ Generate hash value from UUID (0-1) → Weighted selection → Assign topic_category
```

### Execution Result

A `topic_category` column is added to each seed persona, assigning topic categories (transportation, public places, daily life, shopping, school, meals, workplace) according to the theme.

In [16]:
import hashlib

THEME_TO_TOPIC_WEIGHTS = {
    "A_交通・移動": {"交通": 0.60, "公共の場": 0.25, "日常生活": 0.15},
    "B_道具・用途": {"日常生活": 0.35, "買い物": 0.20, "学校": 0.20, "食事": 0.15, "職場": 0.10},
    "C_支払い・お金": {"買い物": 0.40, "交通": 0.25, "公共の場": 0.20, "日常生活": 0.15},
    "D_公共施設・マナー手順": {"公共の場": 0.45, "交通": 0.20, "学校": 0.20, "日常生活": 0.15},
    "E_安全・危険": {"交通": 0.30, "公共の場": 0.30, "日常生活": 0.25, "学校": 0.15},
    "F_生活・家事": {"日常生活": 0.70, "食事": 0.30},
    "N_ニュートラル": {"日常生活": 1.00},
}

def stable_u01(s: str) -> float:
    """uuid等から0-1の安定乱数を作る（毎回同じ）"""
    h = hashlib.md5(s.encode("utf-8")).hexdigest()
    x = int(h[:8], 16)
    return x / 0xFFFFFFFF

def pick_weighted(weights: dict, u: float) -> str:
    acc = 0.0
    last = None
    for k, w in weights.items():
        acc += float(w)
        last = k
        if u <= acc + 1e-12:
            return k
    return last

key_col = "uuid" if ("uuid" in df.columns and df["uuid"].astype(str).str.strip().ne("").any()) else "_attr_key"
if key_col not in df.columns:
    raise ValueError("seedに uuid か _attr_key が必要です")

def assign_topic(row):
    theme = str(row.get("jc_theme", "")).strip()
    weights = THEME_TO_TOPIC_WEIGHTS.get(theme)
    if not weights:
        return "日常生活"  # 想定外は安全側
    u = stable_u01(str(row[key_col]))
    return pick_weighted(weights, u)

seed_jc["topic_category"] = seed_jc.apply(assign_topic, axis=1)

## CSV Text Normalization and Unterminated Quote Detection

Clean and normalize dataframe text in advance to prevent issues with **unterminated quotes** and **control characters** during CSV file output.

### Overview

This code combines 4 normalization processes to clean up data:

1. **Unify newline codes** - Convert CRLF/CR to LF for cross-platform compatibility
2. **Remove control characters** - Delete invisible characters that hinder CSV parsing (preserve tabs and newlines)
3. **Unicode safety** - Handle corrupted characters and isolated surrogates
4. **Quote escaping** - Double quotes as needed (usually handled automatically by csv.writer)

Apply the `clean_cell()` function to all cells in all columns to convert the entire dataframe to a safe state.

### Unterminated Quote Detection

The `is_potential_unterminated_quote()` function warns when the number of double quote occurrences in each row is odd. While not a complete detection, it functions as an inexpensive primary screening before CSV writing and can detect potential syntax errors early.

In [17]:
import re
# ============ 3) テキスト正規化（未終了クォートの温床つぶし） ============
# 改行の正規化：CRLF/CR → LF
def normalize_newlines(s: str) -> str:
    return str(s).replace('\r\n', '\n').replace('\r', '\n')

# 制御文字の除去（タブと改行は保持）
_ctrl_re = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')
def strip_ctrl(s: str) -> str:
    return _ctrl_re.sub('', s)

# 孤立サロゲートや非BMPは基本OKだが、壊れた文字（decode失敗由来）があれば落とす
def safe_unicode(s: str) -> str:
    # そのまま返す（必要なら独自の置換を追加）
    return s

# 内部の ASCII ダブルクォートは csv では "" に二重化されるが、
# 念のためここでも安全側に置換（過剰二重化はしないように一旦 raw を記録）
def escape_quotes_for_csv(s: str) -> str:
    # ここでは何もしなくても csv.writer が二重化するが、
    # 途中で他ツール経由する可能性があるなら下の一行を有効化
    # s = s.replace('"', '""')
    return s

def clean_cell(s: str) -> str:
    s = normalize_newlines(s)
    s = strip_ctrl(s)
    s = safe_unicode(s)
    s = escape_quotes_for_csv(s)
    return s

for c in df.columns:
    df[c] = df[c].map(clean_cell)

# ============ 4) 未終了クォート“になり得る”怪しい行の検出 ============
# ポイント：書き出し前に「各行をカンマで連結したと仮定した文字列」の
# ダブルクォート出現数（"）が“奇数”なら危険（完全検出ではないが安価な一次スクリーニング）
def is_potential_unterminated_quote(row_values):
    # ここでは「生CSV文字列」ではなく“連結仮想文字列”で簡易チェック
    s = ",".join(str(v) if v is not None else "" for v in row_values)
    # すでに "" に二重化していない生の " が含まれている場合に引っ掛けたい
    # ただし上の escape_quotes_for_csv を無効化しているのでここは緩めに総数チェック
    return s.count('"') % 2 == 1  # 奇数なら未終了の可能性

## Filter Unterminated Quotes and Output CSV

Detect and exclude problematic rows with potential unterminated quotes, outputting only clean seed data to CSV file. Apply the `is_potential_unterminated_quote()` function to all rows to identify suspicious rows (odd number of double quotes), and extract only safe rows by logical inversion of the mask. Then remove judgment flag columns used in intermediate processing (`is_neutral`, `_has_definition`, `has_neg_jc`) to clean up the data, and save it in CSV format as the final seed data. This prevents CSV syntax errors and parsing failures, ensuring safety in downstream processing.

**Note**: The variable name is `suspects`, but it actually contains **clean data after excluding** suspicious rows.

In [18]:
suspect_mask = seed_jc.apply(lambda r: is_potential_unterminated_quote(r.tolist()), axis=1)
suspects = seed_jc[~suspect_mask].reset_index(drop=True)
suspects = suspects.drop(
     columns=["is_neutral", "_has_definition",
             "has_neg_jc"],  
)
suspects.to_csv("filtered_personas_balanced_clean_2000_jcommonsense.csv", index=False)

## 🏗️ Define Data Structures

### Data Structure for jcommonsenseqa

In [19]:
# jcommonsenseqa用の構造
class JCommonsenseQAData(BaseModel):
    question: str = Field(
        ..., 
        description="日常生活や常識に関する日本語の質問。文脈や状況を明確に含める。"
    )
    choice0: str = Field(..., description="選択肢0: 自然で妥当な選択肢")
    choice1: str = Field(..., description="選択肢1: 自然で妥当な選択肢")
    choice2: str = Field(..., description="選択肢2: 自然で妥当な選択肢")
    choice3: str = Field(..., description="選択肢3: 自然で妥当な選択肢")
    choice4: str = Field(..., description="選択肢4: 自然で妥当な選択肢")
    answer_index: int = Field(
        ..., 
        ge=0, 
        le=4,
        description="正解の選択肢のインデックス(0-4)"
    )
    reasoning: str = Field(
        ...,
        description="なぜその選択肢が正解なのかの説明"
    )


## 📝 Configuration 1: Using Seed Data

Uses the persona dataset directly as seed,
and Data Designer automatically samples columns from the dataset.


In [71]:
# Seedデータ使用版の設定ビルダー
# 重要: seed_datasetパラメータでDataFrameを渡す
config_builder_with_seed_jcommonsenseqa = DataDesignerConfigBuilder(
    model_configs=model_configs,
)

In [72]:
seed_dataset_reference_jcommonsense = data_designer_client.upload_seed_dataset(
    dataset="filtered_personas_balanced_clean_2000_jcommonsense.csv",
    repo_id="data-designer/filtered_personas_clean_balanced_2000_jcommonsense_add_jc_theme",
    datastore_settings={"endpoint": "http://localhost:3000/v1/hf"},
)

[11:35:31] [INFO] 🔄 Uploading seed dataset to datastore


Upload 0 LFS files: 0it [00:00, ?it/s]

In [73]:
# Pass the reference to the config builder for use during generation.
config_builder_with_seed_jcommonsenseqa.with_seed_dataset(
    seed_dataset_reference_jcommonsense,
)

DataDesignerConfigBuilder(
    seed_dataset: 'data-designer/filtered_personas_clean_balanced_2000_jcommonsense_add_jc_theme/filtered_personas_balanced_clean_2000_jcommonsense.csv'
    seed_dataset_columns: [
        "uuid",
        "professional_persona",
        "sports_persona",
        "arts_persona",
        "travel_persona",
        "culinary_persona",
        "persona",
        "cultural_background",
        "skills_and_expertise",
        "skills_and_expertise_list",
        "hobbies_and_interests",
        "hobbies_and_interests_list",
        "career_goals_and_ambitions",
        "sex",
        "age",
        "marital_status",
        "education_level",
        "occupation",
        "region",
        "area",
        "prefecture",
        "country",
        "age_band",
        "_all_text",
        "_core_text",
        "_core_len",
        "_attr_key",
        "score_finance",
        "score_safety",
        "score_vocab",
        "score_public",
        "score_tools",
        "score_life",
        "score_geo",
        "score_culture",
        "_geo_text",
        "_tools_text",
        "_kw_hits",
        "jc_category",
        "max_score_any",
        "_public_bonus",
        "_religion_pen",
        "jc_theme",
        "topic_category"
    ]
)

### Add jcommonsenseqa Generated Columns

Seed data columns can be directly referenced in the format `{{ column_name }}`.

In [74]:
# jcommonsenseqaデータ生成

# seedデータの列（age, gender, occupation等）を直接参照
config_builder_with_seed_jcommonsenseqa.add_column(
    LLMStructuredColumnConfig(
        name="jcqa_data",
        model_alias=MODEL_ALIAS,
        system_prompt=SYSTEM_PROMPT,
        prompt=(
            "以下のペルソナに基づいて、日本語の常識推論問題を生成してください。\n"
            "ただし、答えが個人の好み・性格・価値観で変わる内容は禁止し、\n"
            "日本で一般的な知識・慣習・手順により答えが1つに定まる問題だけを作成してください。\n\n"
            "【ペルソナ情報（最小）】\n"
            "職業: {{ occupation }}\n"
            "居住地（都道府県）: {{ prefecture }}\n"
            "{% if region is defined %}生活圏（地方）: {{ region }}\n{% endif %}"
            "{% if marital_status is defined %}家族: {{ marital_status }}\n{% endif %}"
            "\n"
            "【状況カテゴリ指定】\n"
            "状況カテゴリは『{{ topic_category }}』としてください。\n\n"
            "【テーマ指定】\n"
            "この問題のテーマは『{{ jc_theme }}』としてください。\n"
            "このテーマと状況カテゴリの両方に整合する日本の一般常識（用途・場所・手順・呼称）で答えが一意に決まる問題にしてください。\n"
            "ただし、両者が矛盾する場合は『{{ jc_theme }}』に整合する内容を優先してください。\n\n"
            "【問題作成の条件】\n"
            "- 質問は具体的な状況を1〜2文\n"
            "- 選択肢は5つ。全て一見もっともらしいが、正解は1つだけ\n"
            "- 正解以外の4つは同カテゴリで意味が近く紛らわしい語を使う\n"
            "- 心理/感情/道徳で割れる内容は禁止（例：嬉しい/悲しい/正しい/許せない/思いやり 等）\n\n"
            "【出力形式（必ずこの形式を守ってください）】\n"
            "question: <質問文>\n"
            "choices:\n"
            "0. <選択肢>\n"
            "1. <選択肢>\n"
            "2. <選択肢>\n"
            "3. <選択肢>\n"
            "4. <選択肢>\n"
            "answer: <0〜4の数字を1文字で出力>\n"
        ),
        output_format=JCommonsenseQAData,
    )
)

print("jcommonsenseqa生成カラムを追加しました")

jcommonsenseqa生成カラムを追加しました


## 🔍 Quality Evaluation Setup

Evaluate data quality using LLM-as-a-Judge

In [75]:
# 品質評価用のルーブリック定義
QuestionClarityRubric = Score(
    name="question_clarity",
    description="質問や状況の明確性の評価",
    options={
        "明確": "質問や状況が具体的で明確に記述されており、意図が容易に理解できる。",
        "やや不明確": "質問や状況は理解できるが、一部曖昧な表現や不足している情報がある。",
        "不明確": "質問や状況の意図が不明確で、追加の説明が必要。",
    },
)

DifficultyRubric = Score(
    name="difficulty",
    description="問題の難易度の評価",
    options={
        "易しい": "日本人の多くが直感的に答えられるレベルの問題。背景知識がほとんど不要。",
        "普通": "一般的な日本人であれば少し考えれば答えられるレベルの問題。基本的な知識や状況理解が必要。",
        "難しい": "ある程度の知識や深い理解、丁寧な読み取りが必要となる問題。人によっては間違えやすい。",
    },
)

# Seedあり版に品質評価を追加
config_builder_with_seed_jcommonsenseqa.add_column(
    LLMJudgeColumnConfig(
        name="quality_metrics",
        model_alias=JUDGE_MODEL_ALIAS,
        prompt=(
            "以下の生成されたデータの品質を評価してください:\n\n"
            # jcqa_dataが存在する場合
            "【jcommonsenseqa問題】\n"
            "質問: {{ jcqa_data.question }}\n"
            "選択肢:\n"
            "0. {{ jcqa_data.choice0 }}\n"
            "1. {{ jcqa_data.choice1 }}\n"
            "2. {{ jcqa_data.choice2 }}\n"
            "3. {{ jcqa_data.choice3 }}\n"
            "4. {{ jcqa_data.choice4 }}\n"
            "正解: {{ jcqa_data.answer_index }}\n"
            "推論: {{ jcqa_data.reasoning }}\n"
            "上記の内容について、日本の文化的適合性、内容の明確性、推論・説明の質を評価してください。"
        ),
        scores=[
            QuestionClarityRubric,
            DifficultyRubric,
        ],
    )
)


# スコアを個別のカラムに抽出
for builder in [config_builder_with_seed_jcommonsenseqa]:
    builder.add_column(
        ExpressionColumnConfig(
            name="clarity_score",
            expr="{{ quality_metrics.question_clarity.score if quality_metrics else 'N/A' }}",
        )
    )
    builder.add_column(
        ExpressionColumnConfig(
            name="difficulty",
            expr="{{ quality_metrics.difficulty.score if quality_metrics else 'N/A' }}",
        )
    )
    
print("品質評価カラムを追加しました")

品質評価カラムを追加しました


## 🔁 Generate Preview

First check quality with a small amount of data

In [77]:
# Seedあり版のプレビュー
print("\n" + "="*0)
print("Seedデータあり版のプレビューを生成中...")
print("="*10)

preview_with_seed_jcommonsenseqa = data_designer_client.preview(
    config_builder_with_seed_jcommonsenseqa,
    num_records=1,
)

print("\nプレビュー生成完了!")
preview_with_seed_jcommonsenseqa.display_sample_record()

[11:35:59] [INFO] ✅ Validation passed
[11:35:59] [INFO] 🚀 Starting preview generation
[11:35:59] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[11:35:59] [INFO] 🩺 Running health checks for models...




Seedデータあり版のプレビューを生成中...


[11:36:00] [INFO]   |-- 👀 Checking 'openai/gpt-oss-120b' in provider named 'nvidiabuild' for model alias 'quality-judge'...
[11:36:00] [INFO]   |-- ✅ Passed!
[11:36:00] [INFO]   |-- 👀 Checking 'openai/gpt-oss-120b' in provider named 'nvidiabuild' for model alias 'gpt-oss-120b'...
[11:36:00] [INFO]   |-- ✅ Passed!
[11:36:01] [INFO] ⏳ Processing batch 1 of 1
[11:36:01] [INFO] 🌱 Sampling 1 records from seed dataset
[11:36:01] [INFO]   |-- seed dataset size: 2000 records
[11:36:01] [INFO]   |-- sampling strategy: ordered
[11:36:01] [INFO] 🗂️ Preparing llm-structured column generation
[11:36:01] [INFO]   |-- column name: 'jcqa_data'
[11:36:01] [INFO]   |-- model config:
{
    "alias": "gpt-oss-120b",
    "model": "openai/gpt-oss-120b",
    "inference_parameters": {
        "temperature": 0.9,
        "top_p": 0.95,
        "max_tokens": 2048,
        "max_parallel_requests": 8,
        "timeout": 1200,
        "extra_body": null
    },
    "provider": "nvidiabuild"
}
[11:36:06] [INFO] 🐙 Pro


プレビュー生成完了!


                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                       ┃ Value                                                                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uuid                       │ 749db6e7c2e245b2ae3b46aa12c4f1e0                                                   │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ professional_persona       │ 中嶋                                                                               │
│                            │ 仁子は保険契約のリスク評価と顧客ニーズの体系的分析に長年従事し、退職後もメンタリ … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sports_persona             │ 中嶋                                                                               │
│                            │ 仁子は季節に合わせたウォーキングとコミュニティの軽運動クラスで体力維持を図り、競 … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ arts_persona               │ 中嶋                                                                               │
│                            │ 仁子は茶道と書道の伝統的稽古を基盤に、デジタル墨絵やインタラクティブ茶室体験とい … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ travel_persona             │ 中嶋                                                                               │
│                            │ 仁子は近郊の歴史的寺院や季節の農産物直売所への日帰り訪問を計画し、列車予約と宿泊 … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ culinary_persona           │ 中嶋                                                                               │
│                            │ 仁子は季節の根菜と海藻を使用した低塩和食を好み、抹茶と煎茶の抽出時間を微調整しな … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ persona                    │ 中嶋                                                                               │
│                            │ 仁子は組織的なリスク管理と健康志向の生活習慣を統合し、オープンマインドと計画性で … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ cultural_background        │ 三重県出身で近畿地方特有の温かい人情と、年長者への敬意を重んじる価値観を持ち、真 … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ skills_and_expertise       │ 保険契約の管理と更新、リスク評価・顧客のニーズ把握、法規制の遵守に加えて、Excelや… │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ skills_and_expertise_list  │ ['保険契約管理', 'リスク評価', '顧客対応', '法規制遵守', 'Excel業務']              │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ hobbies_and_interests      │ 毎朝の散歩と軽い体操で健康維持に努め、地域の健康講座で講師を務めるほか、茶道や書 … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ hobbies_and_interests_list │ ['散歩・体操', '健康講座講師', '茶道', '書道', '新しい表現への探索']               │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ career_goals_and_ambitions │ 退職後も健康保険商品を通じた高齢者の生活支援に関わり、経験を活かした若手代理店へ … │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sex                        │ 女                                                                                 │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ age                        │ 64                

In [78]:
# プレビューの分析を表示
print("\nプレビューデータの分析:")
preview_with_seed_jcommonsenseqa.analysis.to_report()


プレビューデータの分析:


──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1                               │ 48                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                                       ┃          data type ┃                   number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uuid                                              │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ professional_persona                              │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ sports_persona                                    │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ arts_persona                                      │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ travel_persona                                    │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ culinary_persona                                  │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ persona                                           │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ cultural_background                               │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ skills_and_expertise                              │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ skills_and_expertise_list                         │             string │                             1 (100.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ hobbies_and_interests                             │    

In [79]:
# プレビューデータをDataFrameとして確認
preview_df = preview_with_seed_jcommonsenseqa.dataset
print("\nプレビューデータの最初の数件:")
print(preview_df.head())


プレビューデータの最初の数件:
                               uuid  \
0  749db6e7c2e245b2ae3b46aa12c4f1e0   

                                professional_persona  \
0  中嶋 仁子は保険契約のリスク評価と顧客ニーズの体系的分析に長年従事し、退職後もメンタリングと...   

                                      sports_persona  \
0  中嶋 仁子は季節に合わせたウォーキングとコミュニティの軽運動クラスで体力維持を図り、競争的な...   

                                        arts_persona  \
0  中嶋 仁子は茶道と書道の伝統的稽古を基盤に、デジタル墨絵やインタラクティブ茶室体験といった非...   

                                      travel_persona  \
0  中嶋 仁子は近郊の歴史的寺院や季節の農産物直売所への日帰り訪問を計画し、列車予約と宿泊先のロ...   

                                    culinary_persona  \
0  中嶋 仁子は季節の根菜と海藻を使用した低塩和食を好み、抹茶と煎茶の抽出時間を微調整しながら、...   

                                             persona  \
0  中嶋 仁子は組織的なリスク管理と健康志向の生活習慣を統合し、オープンマインドと計画性で高齢者...   

                                 cultural_background  \
0  三重県出身で近畿地方特有の温かい人情と、年長者への敬意を重んじる価値観を持ち、真面目さと健康...   

                                skills_and_expertise  \
0  保険契約の管理と更新、リスク評価・顧客のニーズ把握、法規制の遵守に加えて、ExcelやWor...   



## 🆙 Generate Production Data

If no issues in preview, generate a large-scale dataset

In [ ]:
# Seedあり版の本番生成
NUM_RECORDS = 8000  # 必要に応じて調整

print("\n" + "="*80)
print(f"Seedデータあり版 {NUM_RECORDS}件のデータを生成中...")
print("="*80)

job_with_seed = data_designer_client.create(
    config_builder_with_seed_jcommonsenseqa,
    num_records=NUM_RECORDS,
)

print("ジョブを実行中... 完了を待機しています")

# ジョブ完了を待機
results_with_seed = job_with_seed.wait_until_done()
print("\nSeedあり版の生成完了!")

## 📊 Analyze Results

In [30]:
# Seedあり版の分析
print("\n" + "="*80)
print("Seedデータあり版の分析")
print("="*80)

analysis_with_seed = job_with_seed.load_analysis()
analysis_with_seed.to_report()


Seedデータあり版の分析


──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 7,992                           │ 48                              │ 99.9%                                       │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                                       ┃          data type ┃                   number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uuid                                              │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ professional_persona                              │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ sports_persona                                    │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ arts_persona                                      │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ travel_persona                                    │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ culinary_persona                                  │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ persona                                           │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ cultural_background                               │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ skills_and_expertise                              │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ skills_and_expertise_list                         │             string │                           2000 (25.0%) │
├───────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────┤
│ hobbies_and_interests                             │    

## 📈 Quality Comparison

Compare quality with and without seed data

In [ ]:
# データの読み込み
df_with_seed = job_with_seed.load_dataset()

print(f"Seedあり版のデータ数: {len(df_with_seed)}")

# 品質スコアの集計
def count_scores(df, name):
    print(f"\n{name}の品質スコア分布:")
    
    for metric in ['clarity_score', 'difficulty']:
        if metric in df.columns:
            counts = df[metric].value_counts()
            print(f"\n{metric}:")
            print(counts)
    
    scores = {
        'clarity': df['clarity_score'].value_counts().to_dict() if 'clarity_score' in df.columns else {},
        'difficulty': df['difficulty'].value_counts().to_dict() if 'difficulty' in df.columns else {},
    }
    return scores

scores_with_seed = count_scores(df_with_seed, "Seedあり版")

Seedあり版のデータ数: 7992

Seedあり版の品質スコア分布:

clarity_score:
clarity_score
明確       7974
やや不明確      18
Name: count, dtype: int64[pyarrow]

difficulty:
difficulty
易しい    6984
普通     1007
難しい       1
Name: count, dtype: int64[pyarrow]


## 💾 Save jcommonsenseqa Data

In [34]:
OUTPUT_DIR = "jcommonsenseqa_8000_filter_jcommonsenseqa_seed_2000_temperature_0_9"
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Seedあり版の保存
job_with_seed.download_artifacts(
    output_path=OUTPUT_DIR,
    artifacts_folder_name="with_seed_data",
)

df_with_seed = job_with_seed.load_dataset()

# DataFrameをJSONLで保存（LoRAチューニング用）
df_with_seed.to_json(
    f"{OUTPUT_DIR}/with_seed_data.jsonl",
    orient='records',
    lines=True,
    force_ascii=False
)

print(f"\nデータを '{OUTPUT_DIR}' ディレクトリに保存しました。")
print("\n保存されたファイル:")
for filename in os.listdir(OUTPUT_DIR):
    filepath = os.path.join(OUTPUT_DIR, filename)
    if os.path.isfile(filepath):
        size = os.path.getsize(filepath) / 1024  # KB
        print(f"  - {filename} ({size:.2f} KB)")

[17:52:08] [INFO] 🏺 Downloading artifacts from Job with ID 'job-2qxnztmlrfrkdkjzuxv2kd'
[17:52:10] [INFO] ✅ Artifacts downloaded to jcommonsenseqa_data_output_filter_jcommonsenseqa_seed_adjust_2000_temperature_0_9_remake_metric_8000_blog_check_en/with_seed_data



データを 'jcommonsenseqa_data_output_filter_jcommonsenseqa_seed_adjust_2000_temperature_0_9_remake_metric_8000_blog_check_en' ディレクトリに保存しました。

保存されたファイル:
  - with_seed_data.jsonl (103554.34 KB)


## 📋 Summary

### What We Did
1. ✅ Correctly configured nvidia/Nemotron-Personas-Japan as seed data
2. ✅ Generated data by directly referencing seed data columns
3. ✅ Generated synthetic data for jcommonsenseqa
4. ✅ Created 2 versions: with and without seed data
5. ✅ Quality evaluation using LLM-as-a-Judge
6. ✅ Generated quality comparison report
